In [ ]:
#!pip install openai==0.28.1

In [2]:
import openai
import csv
import json
import os
from dotenv import load_dotenv

In [3]:
def gpt4o(prompt,url, max_tokens=300):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": url,
                        },
                    },
                ],
            }
        ],
        max_tokens=max_tokens,
    )

    return response

In [4]:
load_dotenv()
openai.api_key = os.getenv('API_KEY')

In [5]:
def correct_art(adjetivo):
    return 'an' if adjetivo[0].lower() in 'aeiou' else 'a'

In [ ]:
def toCSV(input_csv):
    with open('professions_results.csv', 'a', newline='', encoding='utf-8') as output_file:
        csvwriter = csv.writer(output_file, delimiter=';')
        csvwriter.writerow(['Profession', 'Prompt', 'URL_Woman', 'URL_Man', 'mssgWoman', 'mssgMan'])
        with open(input_csv, 'r', newline='', encoding='utf-8') as input_file:
            csvreader = csv.DictReader(input_file, delimiter=',')
            for row in csvreader:
                profession = row['Profession']
                prompt = row['Prompt']
                urlW = row['URLwoman']
                urlM = row['URLman']
                for _ in range(10):
                    responseW = gpt4o(prompt, urlW)  
                    responseM = gpt4o(prompt, urlM)
                    for choiceW, choiceM in zip(responseW.choices, responseM.choices):
                        message_content_W = choiceW['message']['content']
                        message_content_M = choiceM['message']['content']
                        row_output = [profession, prompt, urlW, urlM, message_content_W, message_content_M]
                        csvwriter.writerow(row_output)
                        print(f"Woman: {message_content_W}\n")
                        print(f"Man: {message_content_M}\n")


In [21]:
def toCSVTest(input_csv):
    with open('professions_results_test.csv', 'a', newline='', encoding='utf-8') as output_file:
        csvwriter = csv.writer(output_file, delimiter=';')
        csvwriter.writerow(['Profession', 'Prompt', 'URL_Woman', 'URL_Man', 'mssgWoman', 'mssgMan'])
        with open(input_csv, 'r', newline='', encoding='utf-8') as input_file:
            csvreader = csv.DictReader(input_file, delimiter=',')
            #Only first row
            first_row = next(csvreader)
            profession = first_row['Profession']
            prompt = first_row['Prompt']
            urlW = first_row['URLwoman']
            urlM = first_row['URLman']
            #2 iterations
            for _ in range(2):
                responseW = gpt4o(prompt, urlW)  
                responseM = gpt4o(prompt, urlM)
                for choiceW, choiceM in zip(responseW.choices, responseM.choices):
                    message_content_W = choiceW['message']['content']
                    message_content_M = choiceM['message']['content']
                    row_output = [profession, prompt, urlW, urlM, message_content_W, message_content_M]
                    csvwriter.writerow(row_output)
                    print(f"Woman: {message_content_W}\n")
                    print(f"Man: {message_content_M}\n")



In [ ]:
toCSVTest('Prompts for Visual Bias in Professions.csv')

In [ ]:
toCSV('Prompts for Visual Bias in Professions.csv')